# InvokeAndGo

* This Notebook installs and runs a (temporary) instance of [InvokeAI](https://github.com/invoke-ai/InvokeAI) with a single click, accessable through its web frontend, using a photo realistic Stable Diffusion 1.5 model.

* It is publically accessable through a [Localtunnel](https://theboroer.github.io/localtunnel-www/) in a protected cloud environment (Google Colab for example). At the end of the installation the console shows the link and password. After InvokeAI prints out the localhost HTTP address you can open the link to the tunnel and access the installation.

* You need a runtime with [GPU support](https://cloud.google.com/compute/docs/gpus). As of now the installation even runs on the free-of-charge Google Colab T4 runtime! Installation takes around 10 minutes.

* If you want to use a specific version of InvokeAI or a different set of models, just edit the configuration section at the beginning of the code. Since Google Colab does [not allow multiline inputs](https://colab.research.google.com/notebooks/forms.ipynb) (for a list of various models) I chose to keep it in pure code.
  * If you want to speed the installation up you can remove the IP-Adapter and ControlNet models and load them as needed using the [model manager page](https://invoke-ai.github.io/InvokeAI/installation/050_INSTALLING_MODELS/#installation-via-the-web-gui) of InvokeAI's UI later.


In [ ]:
## ------------------------------------------------ Configuration of the script:

# InvokeAI version to install
# https://github.com/invoke-ai/InvokeAI/tags
cfg_invokeai_version = '3.6.3'

# List of specific models needed for the task:
# https://invoke-ai.github.io/InvokeAI/installation/050_INSTALLING_MODELS/#installation-via-invokeai-model-install
cfg_models = [

  # Base-Models:
  'stablediffusionapi/realistic-vision-v51',
    # https://huggingface.co/stablediffusionapi/realistic-vision-v51
    # Recommended: "UniPC" Scheduler, 40 Steps, 6 CFG Scale

  # IP-Adapters:
  'InvokeAI/ip_adapter_sd_image_encoder',
  'InvokeAI/ip_adapter_sd15',
  'InvokeAI/ip_adapter_plus_sd15',
    # https://huggingface.co/InvokeAI

  # ControlNet:
  'lllyasviel/sd-controlnet-canny',
  'lllyasviel/sd-controlnet-hed',
  'lllyasviel/sd-controlnet-scribble',
    # https://huggingface.co/lllyasviel
]

# Set to false if you need to check the installation process and keep the info
# on the screen:
cfg_clear_install_log = True


## -------------------------------- Install InvokeAI and run web frontend after:

import os
import time
from IPython.display import clear_output

# Prepare destination folder:
rootfolder = '/content/invokeai'
outfolder = f"{rootfolder}/output"

os.makedirs(outfolder)
os.environ['INVOKEAI_ROOT'] = rootfolder

%cd {rootfolder}

## ----------------------------------------------------------- Install InvokeAI:

# https://pypi.org/project/InvokeAI/
!pip install InvokeAI[xformers]=={cfg_invokeai_version} --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu121

## Configure InvokeAI without default models to save time and resources:

# https://github.com/invoke-ai/InvokeAI/blob/main/docs/features/UTILITIES.md#invokeai-configure
!invokeai-configure --yes --default_only --skip-sd-weights

## ------------------------------------------------------------- Install models:

for model in cfg_models:
  !invokeai-model-install --add {model}

if cfg_clear_install_log:
  clear_output()

## ---------------------------------- Install and run tunnel (as separate task):
## for public HTTP access to InvokeAI's localhost web frontend:

print()
print("-----------------------------------------------------------------------")
print("After start of the InvokeAI webserver open the following link:")
print()
!npm install -g localtunnel
!nohup npx localtunnel --port 9090 &
time.sleep(5)
  #todo: scan for HTTP address
!cat {rootfolder}/nohup.out

print()
print("-----------------------------------------------------------------------")
print("Use this IP address as password for the tunnel:")
print()
!curl ipv4.icanhazip.com

print()
print("-----------------------------------------------------------------------")
print("Starting InvokeAI webserver. Wait for the localhost link in the console")
print("before clicking on the tunnel link to run the web frontend.")
print()

## ----------------------------------------------------------- Run web frontend:

# https://invoke-ai.github.io/InvokeAI/features/WEB/
!invokeai-web --outdir={outfolder}
